In [57]:
import cv2
import numpy as np,sys

A = cv2.imread('orange.jpg')
B = cv2.imread('apple.jpg')
M = cv2.imread('mask-240.jpg')

# generate Gaussian pyramid for A
G = A.copy()
gpA = [G]
for i in range(2):
    G = cv2.pyrDown(G)
    gpA.append(G)

# generate Gaussian pyramid for B
G = B.copy()
gpB = [G]
for i in range(2):
    G = cv2.pyrDown(G)
    gpB.append(G)

G = M.copy()
gpM = [G]
for i in range(2):
    G = cv2.pyrDown(G)
    gpM.append(G)

# generate Laplacian Pyramid for A
lpA = [gpA[1]]
for i in range(1,0,-1):
    size = (gpA[i-1].shape[1], gpA[i-1].shape[0])
    GE = cv2.pyrUp(gpA[i], dstsize = size)
    L = cv2.subtract(gpA[i-1],GE)
    lpA.append(L)
    
# generate Laplacian Pyramid for B
lpB = [gpB[1]]  
for i in range(1,0,-1):
    size = (gpB[i-1].shape[1], gpB[i-1].shape[0])
    GE = cv2.pyrUp(gpB[i], dstsize = size)
    L = cv2.subtract(gpB[i-1],GE)
    lpB.append(L)

gprM = [gpM[1]]
for i in range(1,0,-1):
    gprM.append(gpM[i-1])
    
# Now add left and right halves of images in each level
LS = []
for la,lb,gm in zip(lpA,lpB,gprM):
    ls = la * gm + lb * (1.0 - gm)
    LS.append(ls)
    
# now reconstruct
ls_ = LS[0]    
for i in range(1,2):
    size = (LS[i].shape[1], LS[i].shape[0])
    ls_ = cv2.pyrUp(ls_, dstsize = size)
    ls_ = cv2.add(ls_, LS[i])

# image with direct connecting each half
cv2.imshow('Pyramid',ls_);
cv2.imwrite('Blending-Image.jpg',ls_)
cv2.waitKey(0)
cv2.destroyAllWindows()